<figure>
<img src="../Imagenes/logo-final-ap.png"  width="80" height="80" align="left"/> 
</figure>

# <span style="color:blue"><left>Aprendizaje Profundo</left></span>

# <span style="color:red"><center>Método entropía cruzada</center></span>

<center>Aprendizaje reforzado</center>

##   <span style="color:blue">Autores</span>

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 

##   <span style="color:blue">Asesora Medios y Marketing digital</span>
 

4. Maria del Pilar Montenegro, pmontenegro88@gmail.com 

## <span style="color:blue">Asistentes</span>

5. Oleg Jarma, ojarmam@unal.edu.co 
6. Laura Lizarazo, ljlizarazore@unal.edu.co 

## <span style="color:blue">Referencias</span>

1. [Alvaro Montenegro y Daniel Montenegro, Inteligencia Artificial y Aprendizaje Profundo, 2021](https://github.com/AprendizajeProfundo/Diplomado)
1. [Maxim Lapan, Deep Reinforcement Learning Hands-On: Apply modern RL methods to practical problems of chatbots, robotics, discrete optimization, web automation, and more, 2nd Edition, 2020](http://library.lol/main/F4D1A90C476A576238E8FE1F47602C67)
1. [Richard S. Sutton, Andrew G. Barto, Reinforcement learning: an introduction, 2nd edition, 2020](http://library.lol/main/6502B74CE247C4CD4D4FB54747AD7C7E)
1. [Praveen Palanisamy - Hands-On Intelligent Agents with OpenAI Gym_ Your Guide to Developing AI Agents Using Deep Reinforcement Learning, 2020](http://library.lol/main/E4FD128CF9B93E0F7A542B053330517A)
1. [Turing Paper 1936](http://www.thocp.net/biographies/papers/turing_oncomputablenumbers_1936.pdf)

## <span style="color:blue">Contenido</span>

* [Introducción](#Introducción)
* [El ambiente CartPole](#El-ambiente-CartPole)
* [El ambiente CartPole](#El-ambiente-CartPole)
* [El método de entropía cruzada en CartPole](#El-método-de-entropía-cruzada-en-CartPole)

## <span style="color:blue">Introducción</span>

En esta lección vamos a entrenar un agente con base en una red neuronal, la cual será una red de clasificación con función de pérdida entropía cruzada.

## <span style="color:blue">La API OpenAI Gym</span>

La API `OpenAI Gym` tiene una rica colección de ambientes para experimentos de aprendizaje reforzado (AR), usando una interfaz unificada.

La clase principal de Gym es *Env* (de environment). Sus métodos y atributos proveen la información necesaria para poder entrenar agentes que interactúen con el medio ambientes. Las piezas más importante disponibles con *Env* son:

Fuente: [Maxim Lapan](http://library.lol/main/2E612EDEF1D325B87C7F5B1FB5542964)
<figure>
<center>
<img src="../Imagenes/RL_components.jpeg" width="300" height="300" align="right"/>
</center>
</figure>

- Un conjunto de `acciones` que se permite sean ejecutadas en el ambiente.
- El tamaño y bordes de las `observaciones` que el ambiente le provee al agente.
- Un método *step* para ejecutar un acción. El método regresa la nueva observación, la `recompensa` y la indicación de si el `episodio` ha terminado (*done*).
- Un método *reset* que retorna al ambiente a su *estado inicial* y entrega la primera observación.

### El espacio de acciones

Las acciones que puede realizar un agente puede ser discretas, continuas o incluso una combinación de ambas. Por ejemplo en un automóvil manejado de manera autónoma puede ser posible oprimir varios botones a la vez (discreto), oprimir el pedal del acelerador o del freno (continuo) y así. El espacio de acciones contiene o modela todas las posibilidades de acción del agente.

### El espacio de observaciones

Las observaciones son piezas de información que el ambiente puede entregar al agente en cualquier momento del tiempo, además de la recompensa. Las observaciones pueden ser muy simples como un arreglo unidimensional de números o  complejos tensores, como las imágenes provenientes de varias cámaras del automóvil.

Es claro que acciones y observaciones tienen similaridades, por lo que Gym dispone de una clase abstracta llamada *Space* que permite derivar espacios de acciones y espacios de observaciones. La siguiente imagen muestra una diagrama de la clase *Space*.

Existen otras subclases de *Space*, pero estas son las más comúnmente utilizadas.

<figure>
<center>
<img src="../Imagenes/jerarquia_clase_Space_Gym.jpeg" width="300" height="300" align="center"/>
</center>
</figure>

Jerarquía de la clase Space de OpenAI Gym. Fuente: [Maxim Lapan](http://library.lol/main/2E612EDEF1D325B87C7F5B1FB5542964)

La clase abstracta *Space* incluye dos métodos relevantes:

* *sample()*: retorna una muestra aleatoria del espacio.
* *contains(x)*: chequea si el argumento es un elemento del dominio del espacio.

Ambos métodos son abstractos y deben ser implementados en las subclases de *Space*.

* La clase *Discrete* representa un conjunto de ítems mutuamente excluyentes, numerados entre 0 y $n-1$. Su único atributo *n*, es una cuenta de los ítems que la clase describe. Por ejemplo. *Discrete(n=4)* puede representar por ejemplo un espacio de acciones de cuatro direcciones para moverse: *[left, right, up, down]*.

* La clase *Box* representa un tensor *n*-dimensional de números racionales con intervalos *[low, high]*. Por ejemplo puede representar el pedal de un acelerador con un valor simple entre 0.0 y 1.0. En pedal sería codificado en tal caso como *Box(low=0.0, high=1.0, shape=(1,), dtype=np.float32)*. Otro ejemplo puede ser la observación de la pantalla en un juego Atari, la cual es RGB de tamaño 210*160: *Box(low=0, high=255, shape=(210, 160, 3), dtype=np.uint8)*.

* La última subclase de *Space* es *Tuple*, la cual permite combinar varias instancias de *Space* juntas. esta subclase permite crear espacios de acciones y observaciones de la complejidad que se desee.

Un ejemplo del uso de tupla puede ser el siguiente. Supongamos que deseamos modelar algunos controles de un automóvil. Para empezar la dirección (el ángulo de la dirección, en realidad), el acelerador y el freno pueder cambiar en cada instante de tiempo. Por otro lado, podemos tener botones discretos para representar la luces direccionales *(apagadas, izquierda, derecha)* y la bocina *(encendida, apagada)*. Entonces, podemos crear *Tupla(spaces=(Box(low=0.0, high=1.0, shape=(3,), dtype=np.float32), Discrete(3), Discrete(2)
)*.

### El ambiente

El ambiente es representado por la clase *Env*. Todo ambiente tiene dos miembros de tipo *Space*: 

* *action_space*: acciones permitidas que entrega el ambiente al agente.
* *observation_space*: observaciones proveidas por el ambiente al agente.

Además el ambiente tiene los siguientes métodos:

* *reset()*: coloca el ambiente en el estado inicial.
* *step()*: Permite al agente tomar una acción y retorna información como respuesta a la acción: la siguiente observación, la recompensa y la bandera indicando si el episodio ha  finalizado. Esta es la pieza central de la funcionalidad del ambiente.

Estas características permiten crear código genérico que podría trabajar con cualquier ambiente.

## <span style="color:blue">El ambiente CartPole</span>

Fuente [OpenAI Gym](https://gym.openai.com/)
<figure>
<center>
<img src="../Imagenes/Ejemplo_polea.gif" width="200" height="200" align="left"/>
</center>
</figure>
    
Este es nuestro primer acercamiento a OpenAI Gym. En este primer  ejemplo conocido como CartPole es un clásico en la teoría de control. En la animación *gif* a la izquierda  observa el objeto de interés. Es una palo, sostenido en la parte inferior por un pasador unido a una base. Ignoramos posibles complicaciones como la fricción. Es decir suponemos que el palo se mueve libremente sobre el pasador. 


Obviamente el palo puede caer muy fácilmente, por lo que el propósito es mover la base hacia la izquierda o hacia la derecha para evitar que el palo caiga. Ese es el problema a resolver. El problema puede ser abordado como un problema de control que lleva a diseñan un modelo en ecuaciones diferenciales. Pero en aprendizaje reforzado buscamos resolver el problema, sin utilizar los conceptos de la física correspondientes. De momento vamos a explorar un poco el  ambiente.

La observación que entrega el ambiente es un arreglo de cuatro posiciones conteniendo respectivamente  *coordenada del centro de masas*, *velocidad*, *ángulo con respecto a la plataforma*, y *velocidad angular*.

El espacio de acciones es de tipo *Discrete* y contiene dos elementos 0 y 1, o *izquierda* y *derecha*.


## <span style="color:blue">El método de entropía cruzada en CartPole</span>

El núcleo de nuestro modelo es una red neuronal(RN) con una capa oculta de de 128 neuronas con un rectificador lineal (ReLU) como función de activación. El tamaño 128 es completamente arbitrario en este ejemplo. Sin embargo nuestro método es muy robusto y converge rápidmente.

Comenzamos definiendo algunas constantes globales:

- HIDDEN_SIZE: tamaño de la capa oculta.
- BATCH SIZE: número de episodios en cada iteración.
- PERCENTILE: porcentaje de recompensas totales que vamos a conservar para determinar episodios *elite*. 

Es decir, vamos a dejar por fuera del entrenamiento de la red, los datos asociados a episodios en donde la recompensa total esté por debajo del 30\% inferior.

Adicionalmente definimos la red neuronal que usaremos.

### Librerías

In [9]:
import gym
from collections import namedtuple
import numpy as np
from torch.utils.tensorboard import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim


### Constantes

In [8]:
HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70

### Red neuronal

In [7]:
# Perceptron con una capa oculta
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)

### Información de los episodios

Vamos a definir dos clases auxiliares para manejar la información de los episodios. Estas son clases tupla de tipo `namedtuple`. Este tipo de tupla definida en el módulo estándard *collections* de Python 3+. Es una tupla en la cual cada elemento recibe un nombre, por el cual puede ser llamado.

In [6]:
# clases namedtuple para manejo de información
# de episodios
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])


*EpisodeStep* se usará para representar la información de un paso simple que el agente hace en un episodio. Tal información es la observación recibida y la acción ejecutado por el agente.

*Episode* mantiene la información global de un episodio: la recompensa total recibida en el episodio y la colección completa de EpisodeStep de tal episodio.

### Como sucede el entrenamiento

El entrenamiento sucederá de la siguiente forma.

1. Se selecciona el 70% de los episodios con más alta recompensa. Estos episodios los llamamos *élite*.
1. De los episodios *élite* se extrae la información de observaciones para la entrada de la red y acción para la salida de la red. La red es de clasificación, por lo que se usará como función de pérdida la entropía cruzada, la cual se calcula directamente de los valores de la salida de la red que denominamos en este caso `logits`. 

#### Objetivo del entrenamiento

Buscamos entrenar la red neuronal, con la información de las observaciones, para que prediga la distribución de probabilidades de la siguiente acción. 

#### Que hace el agente en cada iteración de entrenamiento

1. El agente en cada paso recibe del ambiente la recompensa, la nueva observacion y las posibles acciones disponibles, que este caso siempre son dos: izquierda o derecha.
1. Entonces le entrega a la red la nueva observación y recibe como respuesta de un vector de probabilidades para la siguiente acción.
3. El agente selecciona la siguiente acción de forma aleatoria, usando el modelo de probabilidad recibido.
4. El agente informa al ambiente sobre su siguiente acción.

#### Nota sobre eficiencia

Es más eficiente usar la función de pérdida de entropía cruzada de Pytorch basada en los logits.

### Función de iteración

Esta función genera lotes (batches) de episodios. Esta función es el núcleo de la acción del agente.

In [5]:
# generador de lotes de datos

def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)
    while True:
        obs_v = torch.FloatTensor([obs])
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        step = EpisodeStep(observation=obs, action=action)
        episode_steps.append(step)
        if is_done:
            e = Episode(reward=episode_reward, steps=episode_steps)
            batch.append(e)
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs



Observe que la función *iterate_batches* realmente es un generador. La línea 25 muestra el `yield` de la función, lo que la hace un generador. 

La función ejecuta exáctamente *bath_size* episodios. En nuestro caso será 16.

la función hace los siguiente dentro del ciclo while:

1. Convierte la observación recibida en un tensor. línea 10.
1. Para la observación a la red y recibe la nueva distribución de acciones y al conveirte a un arreglo numpy. Líneas 11 y 12.
1. Selecciona aleatoriamente la siguiente acción con base en el modelo de probabilidad recibido. Línea 13.
1. Entrega la acción al ambiente y éste le retorna la nueva observación, recompensa, la indicación de si el episodio terminó. Se ignora la el vector de posibles acciones, porque no se requiere en este caso. (14).
1. Se acumula la recompensa del paso y se guarda la información del paso  en las tuplas namedtuple (15,16 y 17).
1. Al finalizar el episodio, se guarda la información de todo el episodio en la tupla Episode y se sube a la lista del lote(batch). Se reinician los objetos para un nuevo episodio. Si el lote de episodios se ha completado se entrega el lote y se vacía el lote de episodios. Sino se iniciará un nuevo episodio. (18-27).

### Filtro de lotes de episodios

Como se dijo arriba, la recompensa acumulada de cada episodio se usa como insumo para determinar los episodios élite. El entrenamiento de la red se realiza únicamente con estos episodio elite.

La función *filter_batch* recibe un lote de entrenamiento entregado por el generador de lotes *iterate_batches* y el porcentaje de episodios que se consideran como *élite*.

La función hace lo siguiente:

1. Extrae las recompensas totales de cada episodio de los (batch) de entrenamiento y las coloca en una lista (rewards). Línea 2.
1. Calcula la frontera inferior, para definir los episodios *élite*. Se calcula el percentil 70 de las recompensas. (3).
1. Se calcula la recompensa promedio. Esta recompensa promedio se usará para determinar cuando parar el entrenamiento.
1. Se extrae la observación y la acción de cada episodio *élite* y se suben a listas. (6-12).
1. Se suben las listas a tensores y se retornan, junto con la frontera inferior de recompensa y la recompsnsa promedio.

In [10]:
def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))

    train_obs = []
    train_act = []
    for reward, steps in batch:
        if reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, steps))
        train_act.extend(map(lambda step: step.action, steps))

    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean

### Entrenamiento

Finalmente tenemos el ciclo completo de entrenamiento. Usaremos un objeto SummaryWriter para guardar algunos objetos y verlos en Tensorboard.

Note

In [11]:
if __name__ == "__main__":
    env = gym.make("CartPole-v0")
    #env = gym.wrappers.Monitor(env, directory="mon", force=True)
    obs_size = env.observation_space.shape[0]
    n_actions = env.action_space.n

    net = Net(obs_size, HIDDEN_SIZE, n_actions)
    objective = nn.CrossEntropyLoss()
    optimizer = optim.Adam(params=net.parameters(), lr=0.01)
    writer = SummaryWriter(comment="-cartpole")

    for iter_no, batch in enumerate(iterate_batches(
            env, net, BATCH_SIZE)):
        obs_v, acts_v, reward_b, reward_m = \
            filter_batch(batch, PERCENTILE)
        optimizer.zero_grad()
        action_scores_v = net(obs_v)
        loss_v = objective(action_scores_v, acts_v)
        loss_v.backward()
        optimizer.step()
        print("%d: loss=%.3f, reward_mean=%.1f, rw_bound=%.1f" % (
            iter_no, loss_v.item(), reward_m, reward_b))
        writer.add_scalar("loss", loss_v.item(), iter_no)
        writer.add_scalar("reward_bound", reward_b, iter_no)
        writer.add_scalar("reward_mean", reward_m, iter_no)
        if reward_m > 199:
            print("Resuelto!")
            break
    writer.close()


0: loss=0.659, reward_mean=17.5, rw_bound=17.5
1: loss=0.660, reward_mean=14.3, rw_bound=16.0
2: loss=0.677, reward_mean=19.5, rw_bound=19.5
3: loss=0.682, reward_mean=23.1, rw_bound=28.5
4: loss=0.666, reward_mean=29.6, rw_bound=34.5
5: loss=0.648, reward_mean=31.6, rw_bound=37.0
6: loss=0.649, reward_mean=29.8, rw_bound=31.5
7: loss=0.646, reward_mean=40.1, rw_bound=47.5
8: loss=0.635, reward_mean=41.1, rw_bound=44.0
9: loss=0.606, reward_mean=42.2, rw_bound=56.0
10: loss=0.607, reward_mean=40.5, rw_bound=54.0
11: loss=0.608, reward_mean=57.2, rw_bound=65.0
12: loss=0.594, reward_mean=58.6, rw_bound=86.5
13: loss=0.585, reward_mean=63.3, rw_bound=63.5
14: loss=0.572, reward_mean=64.1, rw_bound=71.5
15: loss=0.583, reward_mean=79.7, rw_bound=89.0
16: loss=0.569, reward_mean=79.7, rw_bound=100.0
17: loss=0.550, reward_mean=70.9, rw_bound=82.0
18: loss=0.543, reward_mean=60.6, rw_bound=71.0
19: loss=0.547, reward_mean=79.1, rw_bound=91.5
20: loss=0.546, reward_mean=80.9, rw_bound=96.5
2

### Abre Tensorboard 

Para explorar como ha ido la pérdidad, la frontera inferior de recompensa y la recompensa promedio.

Si desea ver al actividad del agente descomente la línea 3 para tener un monitor de actividad.

In [ ]:
!tensorboard --logdir=runs


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.6.0 at http://localhost:6006/ (Press CTRL+C to quit)


In [16]:
# Cierra el ambiebte. LA venta X11 se cierra.
env.close()

### Implementacion usando clases

In [29]:
import torch.nn as nn

class Net(nn.Module):
    '''
    Perceptron with a hidden layer
    '''
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )
        
        self.optimizer = None
        self.loss = None
        self.iteration = 0

    def forward(self, x):
        return self.net(x)
    
    def compil(self, optimizer, loss):
        self.optimizer = optimizer
        self.loss = loss

    def fit(self, X, y, epochs=1, verbose=True):
        for epoch in range(epochs):
            self.optimizer.zero_grad()
            predicts = self.forward(X)
            loss = self.loss(predicts, y)
            loss.backward()
            self.optimizer.step()
            self.iteration += 1
            if verbose:
                print("iteration = %d: loss=%.3f " % (self.iteration, loss.item()))
                

In [23]:
import numpy as np
import torch.nn as nn
import torch
from collections import namedtuple

class Agent:
    def __init__(self,  batch_size, percentile):
        self.percentile = percentile
        self.batch_size = batch_size
        self.env = None
        self.net = None
        self.batch = self.iterate_batches()#generator of batches

    def set_net(self, net):
        self.net = net
    
    def set_env(self, env):
        self.env = env
    
    
    def iterate_batches(self):
        Episode = namedtuple('Episode', field_names=['reward', 'steps'])
        EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])
        batch = []
        episode_reward = 0.0
        episode_steps = []
        sm = nn.Softmax(dim=1)
        
        obs = self.env.reset() # first observation
        while True:
            obs_v = torch.FloatTensor([obs])
            act_probs_v = sm(self.net(obs_v)) # compute probabilities for next action
            act_probs = act_probs_v.data.numpy()[0]
            action = np.random.choice(len(act_probs), p=act_probs) # select action
            next_obs, reward, is_done, _ = self.env.step(action) # send action to env
            episode_reward += reward
            step = EpisodeStep(observation=obs, action=action)
            episode_steps.append(step)
            if is_done:
                e = Episode(reward=episode_reward, steps=episode_steps)
                batch.append(e)
                episode_reward = 0.0
                episode_steps = []
                next_obs = self.env.reset()
                if len(batch) == self.batch_size:
                    yield batch
                    batch = []
            obs = next_obs
        
    
    def filter_batch(self):
        batch = next(self.batch) # generate next batch
        rewards = list(map(lambda s: s.reward, batch)) # extract rewards from the batch
        reward_bound = np.percentile(rewards, self.percentile) # extract top percentile reward
        reward_mean = float(np.mean(rewards))

        train_obs = []
        train_act = []
        for reward, steps in batch:
            if reward < reward_bound:
                continue
            train_obs.extend(map(lambda step: step.observation, steps))
            train_act.extend(map(lambda step: step.action, steps))

        train_obs_v = torch.FloatTensor(train_obs)
        train_act_v = torch.LongTensor(train_act)
        return train_obs_v, train_act_v, reward_bound, reward_mean 
    
    def get_batch(self):
        return self.filter_batch()

In [13]:
import gym

class Env:
    def __init__(self, env="CartPole-v0"):
        self.env = gym.make(env)
    
    def step(self, action):
        return self.env.step(action)
    
    def reset(self):
        return self.env.reset()
    
    @property
    def observation_space(self):
        return self.env.observation_space
    
    @property
    def action_space(self):
        return self.env.action_space

In [26]:
class CrossEntropy:
    def __init__(self,  net, agent, env, reward_limit = 199):
        self.net = net
        self.agent = agent
        self.env = env
        self. reward_limit = reward_limit
        
    def play(self):
        obs_size = self.env.observation_space.shape[0]
        n_actions = self.env.action_space.n
               
        X, y, reward_b, reward_m = self.agent.get_batch()
        while reward_m <= self.reward_limit:
            self.net.fit(X, y)
            X, y, reward_b, reward_m = self.agent.get_batch()
        
        print('¡Resuelto!')
        print("reward_mean=%.1f, rw_bound=%.1f" % (reward_m, reward_b))    
        
 

In [30]:
# main
#
# imports
import gym
from collections import namedtuple
import numpy as np
from torch.utils.tensorboard import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim

# env
env = Env(env="CartPole-v0")
OBS_SIZE = env.observation_space.shape[0]
N_ACTIONS = env.action_space.n

HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70


# brain
brain = Net(obs_size=OBS_SIZE , hidden_size=HIDDEN_SIZE, n_actions=N_ACTIONS)
optimizer = optim.Adam(params=brain.parameters(), lr=0.01)
loss = nn.CrossEntropyLoss()
brain.compil(optimizer, loss)



# agent
agent = Agent(batch_size=BATCH_SIZE, percentile=PERCENTILE)
agent.set_net(brain)
agent.set_env(env)

u = CrossEntropy(net=brain, agent=agent, env=env, reward_limit=199)


In [31]:
u.play()

iteration = 1: loss=0.680 
iteration = 2: loss=0.668 
iteration = 3: loss=0.666 
iteration = 4: loss=0.658 
iteration = 5: loss=0.647 
iteration = 6: loss=0.642 
iteration = 7: loss=0.626 
iteration = 8: loss=0.647 
iteration = 9: loss=0.625 
iteration = 10: loss=0.624 
iteration = 11: loss=0.629 
iteration = 12: loss=0.597 
iteration = 13: loss=0.598 
iteration = 14: loss=0.593 
iteration = 15: loss=0.587 
iteration = 16: loss=0.591 
iteration = 17: loss=0.571 
iteration = 18: loss=0.586 
iteration = 19: loss=0.559 
iteration = 20: loss=0.564 
iteration = 21: loss=0.557 
iteration = 22: loss=0.549 
iteration = 23: loss=0.552 
iteration = 24: loss=0.542 
iteration = 25: loss=0.547 
iteration = 26: loss=0.550 
iteration = 27: loss=0.552 
iteration = 28: loss=0.544 
iteration = 29: loss=0.540 
iteration = 30: loss=0.538 
iteration = 31: loss=0.540 
iteration = 32: loss=0.533 
iteration = 33: loss=0.540 
¡Resuelto!
reward_mean=200.0, rw_bound=200.0
